In [1]:
import random

import torch
import torch.nn as nn
from cmws import util
import torch
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    DirectionalLights,
    Materials,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    HardPhongShader,
    TexturesUV,
    TexturesVertex,
    BlendParams,
    softmax_rgb_blend
)
from pytorch3d.structures.meshes import (
    Meshes,
    join_meshes_as_batch,
    join_meshes_as_scene,
)
import numpy as np
from cmws.examples.scene_understanding import data, render
from cmws import util
import matplotlib.pyplot as plt

device = 'cpu'

In [2]:
np.unravel_index(2, (2,2))

(1, 0)

In [3]:
num_blocks = torch.tensor([[[0,1,0],[1,2,0],[0,0,3]]], device=device)
num_rows, num_cols = num_blocks.shape[-2:]

In [4]:
num_samples = util.get_num_elements(num_blocks[:-2])

num_blocks.shape


/om/user/katiemc/continuous_mws/cmws/util.py:324: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return int(torch.tensor(shape).prod().long().item())


torch.Size([1, 3, 3])

In [5]:
nblocks = num_blocks.view(num_samples, num_rows, num_cols)

In [6]:
util.get_num_elements(num_blocks[:-2])

1

In [13]:
num_primitives = 3
primitives = [
    render.Cube(
        "A", torch.tensor([1.0, 0.0, 0.0], device=device), torch.tensor(0.3, device=device)
    ),
    render.Cube(
        "B", torch.tensor([0.0, 1.0, 0.0], device=device), torch.tensor(0.4, device=device)
    ),
    render.Cube(
        "C", torch.tensor([0.0, 0.0, 1.0], device=device), torch.tensor(0.5, device=device)
    ),
][:num_primitives]

num_grid_rows=2
num_grid_cols = 2
stacking_program = []
raw_locations = [] 
num_blocks = [] 
for row in range(num_grid_rows): 
    row_sp = [] 
    row_raw_locs = [] 
    row_num_blocks=[]
    for col in range(num_grid_cols): 
        # Sample
        single_sp = data.sample_stacking_program(
            num_primitives, device, fixed_num_blocks=False
        )
        single_raw_locs = data.sample_raw_locations(single_sp)
        single_num_blocks = torch.tensor(len(single_sp), device=device).long()
        row_sp.append(single_sp.numpy())
        row_raw_locs.append(single_raw_locs.numpy())
        row_num_blocks.append(single_num_blocks.numpy())
    stacking_program.append(np.array(row_sp))
    raw_locations.append(np.array(row_raw_locs))
    num_blocks.append(np.array(row_num_blocks))
#     stacking_program.append(row_sp)
#     raw_locations.append(row_raw_locs)
#     num_blocks.append(row_num_blocks)
    
#     stacking_program.append(torch.Tensor(row_sp))
#     raw_locations.append(torch.Tensor(row_raw_locs))
#     num_blocks.append(torch.Tensor(row_num_blocks))

/home/katiemc/.conda/envs/cmws/lib/python3.7/site-packages/ipykernel_launcher.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/katiemc/.conda/envs/cmws/lib/python3.7/site-packages/ipykernel_launcher.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [14]:
stacking_program[0]

array([array([1, 2, 0]), array([2, 0])], dtype=object)

In [15]:
sp = torch.Tensor(stacking_program)
sp.shape

ValueError: expected sequence of length 3 at dim 2 (got 2)

In [ ]:
num_primitives = 3
primitives = [
    render.Cube(
        "A", torch.tensor([1.0, 0.0, 0.0], device=device), torch.tensor(0.3, device=device)
    ),
    render.Cube(
        "B", torch.tensor([0.0, 1.0, 0.0], device=device), torch.tensor(0.4, device=device)
    ),
    render.Cube(
        "C", torch.tensor([0.0, 0.0, 1.0], device=device), torch.tensor(0.5, device=device)
    ),
][:num_primitives]
# num_primitives = len(primitives)

# Sample
stacking_program = data.sample_stacking_program(
    num_primitives, device, fixed_num_blocks=False
)
raw_locations = data.sample_raw_locations(stacking_program)

In [10]:
raw_locations.shape

AttributeError: 'list' object has no attribute 'shape'

In [4]:
im_size = 32

shape = stacking_program.shape[:-1]
max_num_blocks = stacking_program.shape[-1]
num_elements = util.get_num_elements(shape)
num_channels = 3
num_blocks = torch.tensor(len(stacking_program), device=device).long()

# [num_primitives]
square_size = torch.stack([primitive.size for primitive in primitives])
# [num_primitives, 3]
square_color = torch.stack([primitive.color for primitive in primitives])

# Convert [*shape, max_num_blocks, 3]
locations = render.convert_raw_locations_batched(raw_locations, stacking_program, primitives)

# Flatten
num_blocks_flattened = num_blocks.reshape(num_elements)
stacking_program_flattened = stacking_program.reshape((num_elements, max_num_blocks))
locations_flattened = locations.view((num_elements, max_num_blocks, 3))

imgs = render.render_cubes(num_blocks_flattened, square_size[stacking_program_flattened], square_color[stacking_program_flattened], locations_flattened, im_size)
imgs = imgs.permute(0, 3, 1, 2)
imgs = imgs.view(*[*shape, num_channels, *imgs.shape[-2:]])

ValueError: not enough values to unpack (expected 3, got 1)

In [5]:
num_cubes = num_blocks_flattened
sizes = square_size[stacking_program_flattened]
colors = square_color[stacking_program_flattened]
positions = locations_flattened

In [11]:

im_size = 512

R, T = look_at_view_transform(1.7, 0, 180,
                              at=((0.0, 0.0, -0.5),))
# R, T = look_at_view_transform(3.5, 0, 0,
#                               up=((0.0, 0.0, 0.0),),
#                               at=((0.0, 0.0, -0.5),))
cameras = FoVPerspectiveCameras(device=device, R=R, T=T,
                                )  # fov=45.0)

sigma = 1e-4
gamma = 1e-4


# Settings for rasterizer (optional blur)
# https://github.com/facebookresearch/pytorch3d/blob/1c45ec9770ee3010477272e4cd5387f9ccb8cb51/pytorch3d/renderer/mesh/shader.py
# implements eqs from SoftRasterizer paper
blend_params = BlendParams(sigma=sigma, gamma=gamma) #,background_color=(0.0, 0.0, 0.0))
raster_settings = RasterizationSettings(
    image_size=im_size,  # crisper objects + texture w/ higher resolution
    blur_radius=np.log(1. / 1e-4 - 1.) * blend_params.sigma,
    faces_per_pixel=1,  # increase at cost of GPU memory,
    bin_size=None
)

# Add light from the front
lights = PointLights(device=device, location=[[0.0, 0.0, 3.0]])

# Compose renderer and shader
renderer = MeshRenderer(
    rasterizer=MeshRasterizer(
        cameras=cameras,
        raster_settings=raster_settings
    ),
    shader=SoftPhongShader(
        device=device,
        cameras=cameras,
        lights=lights,
        blend_params=blend_params
    )
)

# create one mesh per elmt in batch

meshes = []
for batch_idx, n_cubes in enumerate(num_cubes):
    # Combine obj meshes into single mesh from rendering
    # https://github.com/facebookresearch/pytorch3d/issues/15
    vertices = []
    faces = []
    textures = []
    vert_offset = 0 # offset by vertices from prior meshes
    for i, (position, size,color) in enumerate(zip(positions[batch_idx, :n_cubes, :], sizes[batch_idx, :n_cubes],
                                                   colors[batch_idx, :n_cubes, :])):
        position = torch.tensor([position[0]+1, position[1], position[2]])
        cube_vertices, cube_faces = render.get_cube_mesh(position, size)
        # For now, apply same color to each mesh vertex (v \in V)
        texture = torch.ones_like(cube_vertices) * color# [V, 3]
        # Offset faces (account for diff indexing, b/c treating as one mesh)
        cube_faces = cube_faces + vert_offset
        vert_offset = cube_vertices.shape[0]
        vertices.append(cube_vertices)
        faces.append(cube_faces)
        textures.append(texture)

    # Concatenate data into single mesh
    vertices = torch.cat(vertices)
    faces = torch.cat(faces)
    textures = torch.cat(textures)[None]  # (1, num_verts, 3)
    textures = TexturesVertex(verts_features=textures)
    # each elmt of verts array is diff mesh in batch
    mesh = Meshes(verts=[vertices], faces=[faces], textures=textures)
    meshes.append(mesh)

batched_mesh = join_meshes_as_batch(meshes)

# Render image
img = renderer(batched_mesh)   # (B, H, W, 4)

# Remove alpha channel and return (B, im_size, im_size, 3)
img = img[:, ..., :3]#.detach().squeeze().cpu().numpy()

NameError: name 'num_cubes' is not defined

In [12]:
plt.imshow(img[0])
#plt.axis('off')

NameError: name 'img' is not defined

In [44]:
a = torch.tensor([position[0], position[1], 10.0])

In [45]:
a

tensor([-6.7806e-03, -3.0000e-01,  1.0000e+01])